In [1]:
import selectors
import socket

mysel = selectors.DefaultSelector()
keep_running = True

def read(connection, mask):
    "Callback for read events"
    global keep_running

    client_address = connection.getpeername()
    print('read({})'.format(client_address))
    data = connection.recv(1024)
    if data:
        # A readable client socket has data
        print('  received {!r}'.format(data))
        connection.sendall(data)
    else:
        # Interpret empty result as closed connection
        print('  closing')
        mysel.unregister(connection)
        connection.close()
        # Tell the main loop to stop
        keep_running = False

def accept(sock, mask):
    "Callback for new connections"
    new_connection, addr = sock.accept()
    print('accept({})'.format(addr))
    new_connection.setblocking(False)
    mysel.register(new_connection, selectors.EVENT_READ, read)

server_address = ('localhost', 65535)
print('starting up on {} port {}'.format(*server_address))
server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.setblocking(False)
server.bind(server_address)
server.listen(5)

mysel.register(server, selectors.EVENT_READ, accept)

while keep_running:
    print('waiting for I/O')
    for key, mask in mysel.select(timeout=1):
        print(key)
        print(mask)
        print(key.data)
        print(key.fileobj)
        callback = key.data
        callback(key.fileobj, mask)

print('shutting down')
mysel.close()

starting up on localhost port 65535
waiting for I/O
waiting for I/O
waiting for I/O
waiting for I/O
waiting for I/O
SelectorKey(fileobj=<socket.socket fd=1236, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 65535)>, fd=1236, events=1, data=<function accept at 0x0000017C16C7DAF0>)
1
<function accept at 0x0000017C16C7DAF0>
<socket.socket fd=1236, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 65535)>
accept(('127.0.0.1', 64822))
waiting for I/O
SelectorKey(fileobj=<socket.socket fd=1272, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 65535), raddr=('127.0.0.1', 64822)>, fd=1272, events=1, data=<function read at 0x0000017C16C7DA60>)
1
<function read at 0x0000017C16C7DA60>
<socket.socket fd=1272, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 65535), raddr=('127.0.0.1', 64822)>
read(('127.0.0.1', 64822))
  received b'This is the

ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine